In [8]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import pandas as pd
import numpy as np
from funpymodeling.exploratory import freq_tbl
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

# Barra lateral - Botón para cerrar sesión
# WIDGET BUTTON UTILIZADO
if st.sidebar.button("Cerrar sesión"):  # 
    st.warning("Has cerrado sesión. Redirigiendo a la página de inicio de sesión...")
    st.session_state.authenticated = False  # Cambia el estado de autenticación
    st.rerun()  # Reinicia la aplicación

# Credenciales de inicio de sesión
USERNAME = 'programador'
PASSWORD = 'datosseattle'

# Función de inicio de sesión
def login():
    st.title("Inicio de Sesión")
    
    # Campos de entrada para el usuario y la contraseña
    username = st.text_input("Nombre de usuario")
    password = st.text_input("Contraseña", type='password')
    
    # Botón para iniciar sesión
    # WIDGET BUTTON UTILIZADO
    if st.button("INGRESAR"):
        if username == USERNAME and password == PASSWORD:
            # Almacenar el estado de autenticación en session_state
            st.session_state.authenticated = True
            st.success("Inicio de sesión exitoso.")
            return True
        else:
            st.error("Nombre de usuario o contraseña incorrectos.")
            return False
    return False

# Comprobar el estado de autenticación al inicio
if 'authenticated' not in st.session_state:
    st.session_state.authenticated = False

# Iniciar sesión
if not st.session_state.authenticated:
    if not login():
        st.stop()  # Detiene la ejecución si no está autenticado

# Definimos la instancia de streamlit
@st.cache_resource

# Cargamos datos
def load_data_Seattle():
    # Carga desde un archivo csv con índice
    data = pd.read_csv('Seattle.csv', encoding='utf-8', index_col=0)

    # Etapa de procesamiento de Datos
    data_cuant = data.copy()

    # Eliminamos las columnas que no necesitamos
    data_cuant.drop(["latitude","longitude","availability_30","last_scraped","source","host_name","host_response_time","host_verifications","neighbourhood_cleansed",
    "property_type","room_type","bathrooms_text","amenities","calendar_last_scraped","bathroom_text_clean","neighbourhood_Category",
    "amenitiesCat","hostTotalListingsCat","accommodatesCat","minimumNightsCategory","maximumNightsCategory","availabilityYearCat","hostYearsCat","hostResponseCat","hostAcceptanceCat","hostVerifCat","bedsCategory",
    "reviewsMonthCat","reviewScoresCat","reviewLocationCat","reviewRatingCat","reviewCleanCat","reviewCommunicationCat"], axis=1, inplace=True)

    # Seleccionar solo columnas numéricas
    data_cuant = data_cuant.select_dtypes(include=[np.number])

    # Encontramos todos los coeficientes de correlación entre las variables
    corr_factors = abs(data_cuant.corr())

    # Variables a analizar
    data_results = data_cuant[['host_response_rate', 'host_acceptance_rate',
                                'host_total_listings_count', 'accommodates',
                                'reviews_per_month', 'price']]

    # Selecciono las columnas tipo numéricas del dataframe
    numeric_data = data_cuant.select_dtypes(['float', 'int'])
    numeric_cols = numeric_data.columns

    # Agregamos el DataFrame original para análisis de variables categóricas
    data_cat = data.copy()

    return data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat

# Cargo los datos obtenidos de la función 'load_data'
data, data_cuant, corr_factors, data_results, numeric_data, numeric_cols, data_cat = load_data_Seattle()

# Funciones para los modelos de las regresiones no lineales
def funcCuad(x, a, b, c):
    return a * x**2 + b * x + c

# Nueva función: Inversa
def funcInversa(x, a):
    return 1 / a * x

# Nueva función: Cuadrática inversa
def funcCuadraticaInversa(x,a):
    return 1 / a * x ** 2

# Título
st.title('ANÁLISIS DE DATOS')
st.header('Regresión No Lineal')

# Contenido para Seattle
st.subheader('Análisis para Seattle')

# WIDGET CHECKBOX UTILIZADO PARA MOSTRAR HEATMAP
heatmap_checkbox = st.sidebar.checkbox(label="Mostrar Heatmap")

if heatmap_checkbox:
    # Heat Map para todas las variables
    heatmap = px.imshow(corr_factors, text_auto=True,
                        title="Este es el Heatmap", width=800, height=600)
    st.plotly_chart(heatmap)

# WIDGET CHECKBOX UTILIZADO PARA MOSTRAR LA MATRIZ DE CORRELACIÓN
matrix_checkbox = st.sidebar.checkbox(label="Mostrar matriz de correlación")

if matrix_checkbox:
    # Se plasma la matriz de correlación 
    st.write(corr_factors)

# Mover el selector "Variables Objetivo" al lado izquierdo (sidebar)
st.sidebar.text('Seleccione una Variable Objetivo para graficar')
# WIDGET SELECT BOX PARA SELECCIONAR VARIABLES OBJETIVO
vars_result_selected = st.sidebar.selectbox(label='Variables Objetivo', options=data_results.columns)

# WIDGET MULTISELECT BOX PARA SELECCIONAR LAS FÓRMULAS DE CORRELACIÓN NO LINEAL 
st.sidebar.subheader('Correlaciones No Lineales - para Variables Objetivo')
formulas_seleccionadas = st.sidebar.multiselect(
    label='Seleccione las fórmulas para la correlación no lineal', 
    options=['Cuadrática','Inversa','Cuadrática Inversa']
)

st.sidebar.subheader('Criterios de Graficación')
# WIDGET SELECT BOX PARA SELECCIONAR VARIABLE X
x_var = st.sidebar.selectbox(label='Seleccione Variable X', options=numeric_cols)

# Bucle para aplicar las fórmulas seleccionadas
for formula in formulas_seleccionadas:
    if formula == 'Cuadrática':
        # Cálculo para la función cuadrática
        data_copy = data_cuant.copy()
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]
        st.text('Función Cuadrática: y = ax^2 + bx + c')
        try:
            # Ajustamos los parámetros de la función curve_fit
            parametros, covs = curve_fit(funcCuad, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)
            y_pred = funcCuad(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Calculo del coeficiente de determinación del modelo
            r2 = abs(r2_score(y, y_pred))

            # Calculo del coeficiente de correlación
            r = np.sqrt(abs(r2))

            # Mostramos los coeficientes
            st.text(f'Coeficiente de Correlación: {r}')
            st.text(f'Coeficiente de Determinación (R²): {r2}')

            # SCATTERPLOT
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Cuadrática")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la gráfica para la función Cuadrática')

    

    elif formula == 'Inversa':
        # Cálculo para la función Inversa
        data_copy = data_cuant.copy()
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]
        st.text('Función Inversa: y = 1 / a * x')
        try:
            # Ajustamos los parámetros de la función curve_fit
            parametros, covs = curve_fit(funcInversa, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)
            y_pred = funcInversa(x, *parametros)
            data_copy['y_pred'] = y_pred

            # Cálculo del coeficiente de determinación del modelo
            r2 = abs(r2_score(y, y_pred))

            # Cálculo del coeficiente de correlación
            r = np.sqrt(abs(r2))

            # Mostramos los coeficientes
            st.text(f'Coeficiente de Correlación: {r}')
            st.text(f'Coeficiente de Determinación (R²): {r2}')

            # SCATTERPLOT
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Inversa")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la gráfica para la función Inversa')

    # Cuadrática inversa
    elif formula == 'Cuadrática Inversa':
        # Cálculo para la función Cuadrática Inversa
        data_copy = data_cuant.copy()
        x = data_cuant[[str(x_var)]]
        y = data_cuant[[str(vars_result_selected)]]
        st.text('Función Cuadrática Inversa: y = 1 / a * x ** 2')
        try:
            # Ajustamos los parámetros de la función curve_fit
            parametros, covs = curve_fit(funcCuadraticaInversa, data_cuant[x_var], data_cuant[vars_result_selected], maxfev=5000)
            y_pred = funcCuadraticaInversa(x, *parametros)
            data_copy['y_pred'] = y_pred
            
            # Cálculo del coeficiente de determinación del modelo
            r2 = abs(r2_score(y, y_pred))

            # Cálculo del coeficiente de correlación
            r = np.sqrt(abs(r2))
            
            # Mostramos los coeficientes
            st.text(f'Coeficiente de Correlación: {r}')
            st.text(f'Coeficiente de Determinación (R²): {r2}')
            
            # WIDGET SCATTERPLOT PARA VISUALIZAR LOS RESULTADOS DE LAS REGRESIONES NO LINEALES
            # Scatterplot
            figureRNL = px.scatter(data_frame=data_copy, x=x_var, y=[vars_result_selected, 'y_pred'],
                                    title="Regresión No Lineal - Cuadrática Inversa")
            st.plotly_chart(figureRNL)
        except:
            st.text('Es imposible formular la gráfica para la función Cuadrática Inversa')


Overwriting app.py
